<a href="https://colab.research.google.com/github/ardilapablo/VisualizacionDeDatos/blob/main/EjerciciosClase/CLASE2_VISUALIZACION_DE_DATOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
import pandas as pd
import plotly.express as px

# Ruta completa al archivo CSV
ruta_csv = '/content/drive/My Drive/datos_ejercicio_ventas.csv'

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv(ruta_csv)
df.head()


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


Vamos a limpiar nuestros datos eliminando duplicados y filtrando los valores negativos en la columna AMOUNT para que solo cuente con los positivos o 0, ya que un valor negativo en este campo no tendría sentido en un contexto de ventas o predicciones. Decidimos conservar los valores nulos en las columnas FORECAST y FORECAST_YEAR porque corresponden a datos reales (SCENARIO = "Actual") que no necesitan predicción o año de predicción.

In [90]:
# Eliminamos duplicados
df_limpio = df.drop_duplicates()

# Filtramos filas con valores negativos en 'AMOUNT'
df_limpio = df_limpio[df_limpio['AMOUNT'] >= 0]
df_limpio.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


In [91]:
# Contamos cuántos datos hay de cada tipo en la columna 'SCENARIO'
conteo_scenario = df['SCENARIO'].value_counts().reset_index()

# Renombramos las columnas para que sean más descriptivas
conteo_scenario.columns = ['SCENARIO', 'COUNT']

fig = px.bar(conteo_scenario, x='SCENARIO', y='COUNT', title='Cantidad de Actuals vs Forecasts',
             labels={'SCENARIO': 'Tipo de Dato', 'COUNT': 'Cantidad'})

fig.show()

Podemos observar la cantidad de predicciones (AI_forecast) en comparación con los valores reales (actual) en el conjunto de datos. Podemos ver que hay muchas más predicciones que valores actuales.

In [92]:
# Contamos cuántos datos hay de cada sub-marca en la columna SUBBRAND
conteo_subbrand = df['SUBBRAND'].value_counts().reset_index()

# Renombramos las columnas para que sean más descriptivas
conteo_subbrand.columns = ['SUBBRAND', 'COUNT']

fig = px.bar(conteo_subbrand, x='SUBBRAND', y='COUNT', title='Cantidad de Datos por Submarca',
             labels={'SUBBRAND': 'Submarca', 'COUNT': 'Cantidad'})

fig.show()


Podemos ver que Pepsi Max es una de las sub-marcas con mayor volumen de datos mientras que Mountain Dew es la que menos cantidad de datos tiene en la base de datos.

In [93]:
# Agrupamos por submarca y sumamos el AMOUNT
df_amount_subbrand = df.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

fig = px.bar(
    df_amount_subbrand,
    x='SUBBRAND',
    y='AMOUNT',
    title="Importe Total de Ventas (AMOUNT) por Submarca",
    labels={'AMOUNT': 'Monto Total de Ventas', 'SUBBRAND': 'Submarca'},
    color='SUBBRAND'
)

fig.update_layout(
    plot_bgcolor="white",
    paper_bgcolor="white",
    xaxis_title="Submarca",
    yaxis_title="Monto Total (AMOUNT)",
    showlegend=False
)

fig.show()



In [94]:
fig = px.bar(
    df_monthly_counts,
    x='DATE',
    y='COUNT',
    title="Cantidad de Datos por Mes",
    labels={'COUNT': 'Cantidad de Datos', 'DATE': 'Fecha'}
)

fig.show()


##Distribuir las Ventas por País


In [95]:

fig_sales_by_country = px.histogram(
    df,
    x='COUNTRY',
    y='AMOUNT',
    color='SUBBRAND',
    title="Distribución de Ventas Totales por País y Marca",
    labels={'AMOUNT': 'Ventas Totales', 'COUNTRY': 'País'},
    barmode='stack'
)

fig_sales_by_country.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_sales_by_country.show()


##Evolución Mensual de las Ventas por Año




In [96]:


# Asegurarse de que la columna 'DATE' sea de tipo datetime
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupamos por año y mes, calculando el total de AMOUNT para cada mes en cada año
df_monthly_amount = df.groupby(['YEAR', 'MONTH']).agg({'AMOUNT': 'sum'}).reset_index()

fig = px.line(
    df_monthly_amount,
    x='MONTH',
    y='AMOUNT',
    color='YEAR',
    title="Evolución Mensual del Importe (AMOUNT) por Año",
    labels={'AMOUNT': 'Monto Total (AMOUNT)', 'MONTH': 'Mes', 'YEAR': 'Año'},
)

# Ajustamos el eje x para mostrar los meses de manera correcta
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
    )
)

fig.show()


##Distribución de Ventas por Marca

In [97]:


fig_sales_by_brand = px.bar(
    df,
    x='SUBBRAND',
    y='AMOUNT',
    title="Distribución de Ventas Totales por Submarca",
    labels={'AMOUNT': 'Monto Total de Ventas (AMOUNT)', 'SUBBRAND': 'Submarca'},
    color='SUBBRAND',  # Color por cada sub-marca

)

# Cambiar el fondo a blanco
fig_sales_by_brand.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis_title="Submarca",
    yaxis_title="Monto Total (AMOUNT)",
    legend_title_text='Submarca'
)

fig_sales_by_brand.show()


##Tendencia y Estacionalidad del País con Menos Ventas

In [98]:


# Identificamos el país con menos ventas
country_least_sales = df.groupby('COUNTRY')['AMOUNT'].sum().idxmin()

# Filtramos los datos para el país con menos ventas
df_country_least_sales = df[df['COUNTRY'] == country_least_sales]

# Agrupamos por año y mes para obtener la venta total por mes
df_country_least_sales = df_country_least_sales.groupby(['YEAR', 'MONTH']).agg({'AMOUNT': 'sum'}).reset_index()

# Creamos la columna de fecha completa en nivel mensual
df_country_least_sales['DATE'] = pd.to_datetime(df_country_least_sales[['YEAR', 'MONTH']].assign(DAY=1))

# Ordenamos por fecha
df_country_least_sales = df_country_least_sales.sort_values(by='DATE')

# Descomposición de la serie temporal para obtener la estacionalidad
decompose_result = seasonal_decompose(df_country_least_sales['AMOUNT'], model='additive', period=12)

fig_trend = px.line(x=df_country_least_sales['DATE'], y=decompose_result.trend,
                    title=f"Tendencia de Ventas Mensuales en {country_least_sales}",
                    labels={'x': 'Fecha', 'y': 'Tendencia (AMOUNT)'})
fig_trend.show()

fig_seasonal = px.line(x=df_country_least_sales['DATE'], y=decompose_result.seasonal,
                       title=f"Estacionalidad de Ventas Mensuales en {country_least_sales}",
                       labels={'x': 'Fecha', 'y': 'Estacionalidad (AMOUNT)'})

# Añadimos líneas verticales cada 12 meses para resaltar la estacionalidad
for month in range(0, len(df_country_least_sales['DATE']), 12):
    fig_seasonal.add_vline(x=df_country_least_sales['DATE'].iloc[month], line_dash="dash", line_color="red")

fig_seasonal.show()


## Tendencia y Estacionalidad de la Marca con Más Ventas

In [99]:
# Identificamos la marca con más ventas
brand_most_sales = df.groupby('SUBBRAND')['AMOUNT'].sum().idxmax()

# Filtramos los datos para la marca con más ventas
df_brand_most_sales = df[df['SUBBRAND'] == brand_most_sales]

# Agrupamos por año y mes para obtener la venta total por mes
df_brand_most_sales = df_brand_most_sales.groupby(['YEAR', 'MONTH']).agg({'AMOUNT': 'sum'}).reset_index()

# Creamos la columna de fecha completa en nivel mensual
df_brand_most_sales['DATE'] = pd.to_datetime(df_brand_most_sales[['YEAR', 'MONTH']].assign(DAY=1))

# Ordenamos por fecha
df_brand_most_sales = df_brand_most_sales.sort_values(by='DATE')

# Descomposición de la serie temporal para obtener la estacionalidad
decompose_result_brand = seasonal_decompose(df_brand_most_sales['AMOUNT'], model='additive', period=12)

fig_trend_brand = px.line(x=df_brand_most_sales['DATE'], y=decompose_result_brand.trend,
                          title=f"Tendencia de Ventas Mensuales de la Marca {brand_most_sales}",
                          labels={'x': 'Fecha', 'y': 'Tendencia (AMOUNT)'})
fig_trend_brand.show()

fig_seasonal_brand = px.line(x=df_brand_most_sales['DATE'], y=decompose_result_brand.seasonal,
                             title=f"Estacionalidad de Ventas Mensuales de la Marca {brand_most_sales}",
                             labels={'x': 'Fecha', 'y': 'Estacionalidad (AMOUNT)'})

for month in range(0, len(df_brand_most_sales['DATE']), 12):
    fig_seasonal_brand.add_vline(x=df_brand_most_sales['DATE'].iloc[month], line_dash="dash", line_color="red")

fig_seasonal_brand.show()


In [100]:


#Calculamos el error medio por combinación de 'SUBBRAND', 'YEAR', y 'MONTH'
spain_comparison_mean_error = spain_comparison.groupby(['SUBBRAND', 'YEAR', 'MONTH']).agg({
    'RELATIVE_ERROR': 'mean'
}).reset_index()

fig = px.bar(
    spain_comparison_mean_error,
    x='MONTH',
    y='RELATIVE_ERROR',
    color='SUBBRAND',
    title="Error Relativo Promedio de las Predicciones por Submarca y Mes",
    labels={'RELATIVE_ERROR': 'Error Relativo Promedio (%)', 'MONTH': 'Mes'}
)

fig.update_layout(
    xaxis_title="Mes",
    yaxis_title="Error Relativo Promedio (%)",
    legend_title="Submarca"
)

fig.show()


In [101]:

#boxplot
fig = px.box(
    spain_comparison_mean_error,
    x='MONTH',
    y='RELATIVE_ERROR',
    color='SUBBRAND',
    title="Distribución del Error Relativo Promedio de las Predicciones por Submarca y Mes",
    labels={'RELATIVE_ERROR': 'Error Relativo Promedio (%)', 'MONTH': 'Mes'}
)

fig.update_layout(
    xaxis_title="Mes",
    yaxis_title="Error Relativo Promedio (%)",
    legend_title="Submarca"
)

fig.show()
